# Reconstitution INPI

## Recreation du fichier INPI 

In [ ]:
import os
from tqdm import tqdm
import pandas as pd
path = r'C:\Users\PERNETTH\Documents\Projects\InseeInpi_matching\Notebooks_matching\programme_matching\data\output'
list_issue = []
df_matched = pd.DataFrame()
for root, dirs, files in os.walk(path):
    # Test-> on veut pas avant mai 2018
    for name in tqdm(files):
        if name.endswith((".gz")):
            path_gz = '{}\{}'.format(root, name)
            
            df_matched = df_matched.append(
            pd.read_csv(path_gz, compression = 'gzip',low_memory = False))

In [ ]:
path_inpi = r'C:\Users\PERNETTH\Documents\Projects\InseeInpi_matching\Notebooks_matching\programme_matching\data\input\inpi_etb_2_4979272.gz'
df_inpi = pd.read_csv(path_inpi,compression = 'gzip',low_memory = False)

In [ ]:
df_inpi.shape[0] - df_matched.shape[0]

In [ ]:
df_matched_full = df_inpi.merge(df_matched[['index', 'siret', 'ncc', 'Adress_new']], on = 'index',how = 'inner')

In [ ]:
df_matched_full.to_csv('data\\inpi_etb_test_{}.gz'.format(
    df_matched_full.shape[0]),
                       compression='gzip', index = False) 

## Creation base SQL pour App

### Save INSEE

In [ ]:
from inpi_insee import siretisation
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
%load_ext autoreload
%autoreload 2

In [ ]:
param = {
    'communes_insee': r'data\input\communes_france.csv',
    'upper_word':r'data\input\upper_stop.csv',
    'insee': r'data\input\insee_2017_7480120.gz',
    'inpi_etb': r'data\input\inpi_etb_4979272.gz',
}
al_siret = siretisation.siretisation_inpi(param)

In [ ]:

insee_dtype = {
    'siren': 'object',
    'siret': 'object',
    'dateCreationEtablissement': 'object',
    "etablissementSiege": "object",
    "etatAdministratifEtablissement": "object",
    'complementAdresseEtablissement': 'object',
    'numeroVoieEtablissement': 'object',
    'indiceRepetitionEtablissement': 'object',
    'typeVoieEtablissement': 'object',
    'libelleVoieEtablissement': 'object',
    'codePostalEtablissement': 'object',
    'libelleCommuneEtablissement': 'object',
    'libelleCommuneEtrangerEtablissement': 'object',
    'distributionSpecialeEtablissement': 'object',
    'codeCommuneEtablissement': 'object',
    'codeCedexEtablissement': 'object',
    'libelleCedexEtablissement': 'object',
    'codePaysEtrangerEtablissement': 'object',
    'libellePaysEtrangerEtablissement': 'object',
    'count_initial_insee': 'int'
}

insee_col = ['siren',
             'siret',
             'dateCreationEtablissement',
             "etablissementSiege",
             "etatAdministratifEtablissement",
             'complementAdresseEtablissement',
             'numeroVoieEtablissement',
             'indiceRepetitionEtablissement',
             'typeVoieEtablissement',
             'libelleVoieEtablissement',
             'codePostalEtablissement',
             'libelleCommuneEtablissement',
             'libelleCommuneEtrangerEtablissement',
             'distributionSpecialeEtablissement',
             'codeCommuneEtablissement',
             'codeCedexEtablissement',
             'libelleCedexEtablissement',
             'codePaysEtrangerEtablissement',
             'libellePaysEtrangerEtablissement',
             'count_initial_insee']
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}

insee = al_siret.import_dask(file=r'data\input\insee_2017_7480120.gz',
                             usecols=insee_col, dtype=insee_dtype)

list(insee)

In [ ]:
conn = sqlite3.connect('insee.db')
c = conn.cursor()
c.execute("CREATE TABLE INSEE (siren,siret,dateCreationEtablissement, \
 etablissementSiege,complementAdresseEtablissement,numeroVoieEtablissement,  \
 indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,  \
 codePostalEtablissement,libelleCommuneEtablissement,  \
 libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,  \
 codeCommuneEtablissement,codeCedexEtablissement, libelleCedexEtablissement,  \
 codePaysEtrangerEtablissement, libellePaysEtrangerEtablissement,  \
 etatAdministratifEtablissement,count_initial_insee)") 
conn.commit()

insee.compute().to_sql('INSEE', conn, if_exists='replace', index = False)

### Save INPI

In [ ]:
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}


inpi = al_siret.import_dask(file=r'data\inpi_etb_test_4911141.gz',
                       usecols=inpi_col, dtype=inpi_dtype)

In [ ]:
c.execute("CREATE TABLE SIREN (Code Greffe,Nom_Greffe,Numero_Gestion,siren,Type, \
 Siège_PM,RCS_Registre,Adresse_Ligne1,Adresse_Ligne2,Adresse_Ligne3,  \
 Code_Postal, Ville,Code_Commune,Pays,Domiciliataire_Nom,Domiciliataire_Siren,  \
 Domiciliataire_Greffe,Domiciliataire_Complément,Siege_Domicile_Représentant,Nom_Commercial,  \
 Enseigne,Activité_Ambulante, Activité_Saisonnière,Activité_Non_Sédentaire,  \
 Date_Début_Activité,Activité, Origine_Fonds,Origine_Fonds_Info,  \
 Type_Exploitation,ID_Etablissement,Date_Greffe,Libelle_Evt, count, siret, ncc, Adress_new)") 
conn.commit()

inpi.drop(columns = 'index').compute().to_sql('SIREN', conn, if_exists='replace', index = False)

### SAve INPI original

In [ ]:
list(pd.read_csv(
    r'data\input\inpi_etb_cleaned_4979272.gz', 
    compression = 'gzip'))

In [ ]:
inpi = al_siret.import_dask(
    file=r"data\input\inpi_etb_cleaned_4979272.gz",
    usecols=[
        "siren",
        "Type",
        "Adresse_Ligne1",
        "Adresse_Ligne2",
        "Adresse_Ligne3",
        "Code_Postal",
        "Ville",
        "Code_Commune",
        "Pays",
        "count_initial_inpi",
        "index",
        "ncc",
    ],
    dtype={
        "siren": "object",
        "Type": "object",
        "Adresse_Ligne1": "object",
        "Adresse_Ligne2": "object",
        "Adresse_Ligne3": "object",
        "Code_Postal": "object",
        "Ville": "object",
        "Code_Commune": "object",
        "Pays": "object",
        "count_initial_inpi": "object",
        "index": "object",
        "ncc": "object",
    },
)

In [ ]:
conn = sqlite3.connect('inpi_origine.db')
c = conn.cursor()
c.execute("CREATE TABLE INPI_ORIGINE (siren,Type,  Adresse_Ligne1,\
 Adresse_Ligne2,Adresse_Ligne3,Code_Postal, Ville,\
 Code_Commune, Pays,count_initial_inpi,ncc)") 
conn.commit()

inpi.drop(columns = 'index').compute().to_sql('INPI_ORIGINE', conn, if_exists='replace', index = False)

## Test SQL

In [ ]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(r"C:\Users\PERNETTH\Documents\Projects" \
    r"\InseeInpi_matching\Notebooks_matching" \
    r"\programme_matching\App\SQL\inpi_origine.db")
c = conn.cursor()
query = '''SELECT * FROM INPI_ORIGINE WHERE siren = {} '''.format('780129987')
c.execute(query)
df = pd.DataFrame(c.fetchall(), columns= [
        "siren",
        "Type",
        "Adresse_Ligne1",
        "Adresse_Ligne2",
        "Adresse_Ligne3",
        "Code_Postal",
        "Ville",
        "Code_Commune",
        "Pays",
        "count_initial_inpi",
        "ncc",
    ])
df